In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder , StandardScaler, OneHotEncoder
import pickle

In [67]:
#Load Dataset
data=pd.read_csv('Churn_Modelling.csv')

In [68]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [69]:
data.drop(columns=['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [70]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [71]:
#Encoding Catergorical Variable
lable_encoder_gender=LabelEncoder()
data['Gender']=lable_encoder_gender.fit_transform(data['Gender'])

In [72]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [73]:
data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [74]:
"""Since ANN is all about to mathamatical calculations, for data of Geography column has only 3 unique countries if we use Label encoding 
then we will be getting values ['France' as 0, 'Spain' as 1, 'Germany' as 2] ,in ANN it might treat Germany is greater than Spain and 
spain is greater than France..To avoid this we will use One-Hot Encoding for Geography"""

"Since ANN is all about to mathamatical calculations, for data of Geography column has only 3 unique countries if we use Label encoding \nthen we will be getting values ['France' as 0, 'Spain' as 1, 'Germany' as 2] ,in ANN it might treat Germany is greater than Spain and \nspain is greater than France..To avoid this we will use One-Hot Encoding for Geography"

In [75]:
#One Hot Encoding for 'Geography'
onehot_encoder_geo=OneHotEncoder(sparse_output=False)
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])

In [76]:
geo_encoder


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [77]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [78]:
df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [79]:
df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [80]:
#combain one hot encoding output with orginal Data
data=pd.concat([data,df],axis=1)

In [81]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,France,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,France,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [83]:
data.drop(columns=['Geography'],axis=1,inplace=True)

In [84]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
##Save the Gender encoder
with open('lable_encoder_gender.pkl','wb') as file:
    pickle.dump(lable_encoder_gender,file)

In [87]:
##Save the Gender encoder
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [89]:
# dividing the data set into Independent and Dependent features
X=data.drop('Exited',axis=1)
y=data['Exited']

In [90]:
#Train-test split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)

In [91]:
##Scaling the features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [101]:
(X_train.shape[1],)

(12,)

In [97]:
#creating the pickle file for Scaler 
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [114]:
#### ANN Implementation

In [115]:
import tensorflow


In [117]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
import datetime

In [124]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [125]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                832       
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [139]:
##Optimizers and Loss
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.binary_crossentropy

In [141]:
#model compilation
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [142]:
earlyStoppingcallback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [143]:
log_dir="log/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [144]:
tensorboardCallback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [145]:
#Training the Model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[earlyStoppingcallback,tensorboardCallback])

Epoch 1/100


250/250 [==============================] - 2s 3ms/step - loss: 0.4023 - accuracy: 0.8325 - val_loss: 0.3632 - val_accuracy: 0.8515
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3530 - accuracy: 0.8565 - val_loss: 0.3439 - val_accuracy: 0.8610
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3448 - accuracy: 0.8571 - val_loss: 0.3532 - val_accuracy: 0.8605
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3430 - accuracy: 0.8602 - val_loss: 0.3426 - val_accuracy: 0.8595
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3412 - accuracy: 0.8600 - val_loss: 0.3412 - val_accuracy: 0.8555
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3355 - accuracy: 0.8609 - val_loss: 0.3418 - val_accuracy: 0.8645
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3353 - accuracy: 0.8619 - val_loss: 0.3482 - val_accuracy: 0.85

In [148]:
model.save('model.h5')

d:\Restart\ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [150]:
%load_ext tensorboard